In [2]:
import os
import re
import nltk


In [81]:
#define the Stanford NER tagger
st = nltk.tag.StanfordNERTagger('C:/Program Files/stanford-ner-4.2.0/stanford-ner-2020-11-17/classifiers/english.muc.7class.distsim.crf.ser.gz', 'C:/Program Files/stanford-ner-4.2.0/stanford-ner-2020-11-17/stanford-ner.jar', encoding='utf-8')

In [4]:
#config
PATH_DATASET = "dataset"
PATH_INPUT = "processed"
PATH_STORIES = "stories"

PATH_COLLECTION = "3_the_return_of_sherlock_holmes"
FOLDER_PATH = os.path.join('..',PATH_DATASET,PATH_INPUT,PATH_STORIES,PATH_COLLECTION)
f = "the_adventure_of_black_peter.txt"
filepath = os.path.join(FOLDER_PATH,f)


In [78]:
if os.path.isfile(filepath) and f.endswith(".txt") :
    print(f"Process file: {filepath}")

    with open(filepath, "r") as fp:
        content = fp.read()
            
tokenized_content = nltk.tokenize.word_tokenize(content)
tagged_content = st.tag(tokenized_content)

pos_tags = nltk.pos_tag(tokenized_content)

#print(tagged_content[:300])

Process file: ..\dataset\processed\stories\3_the_return_of_sherlock_holmes\the_adventure_of_black_peter.txt


In [77]:
for t in tagged_content:
    if t[1] == "TIME":
        print(t[0])

last
night
night
morning
night
night
last
night
last
night


In [79]:
zipped = [[name, pos, tp[1]] for (name,pos), tp in zip(pos_tags, tagged_content)]
tree = IOB_to_tree(zipped)

person = []
location = []
time = []

for chunk in tree:
    if hasattr(chunk, 'label'):
        entity = " ".join(part[0] for part in chunk.leaves())
        if chunk.label() == "PERSON":
            person.append(entity)
        elif chunk.label() == "LOCATION":
            location.append(entity)
        elif (chunk.label() == "DATE") or (chunk.label() == "TIME"):
            time.append(entity)
        #print(chunk.label(), entity)
print(f"characters: {person}")  
print(f"locations: {location}")
print(f"dates / times: {time}")

characters: ['Wilson', 'Peter Carey', 'Sherlock Holmes', 'Holmes', 'Holmes', 'Watson', 'Allardyce', 'Hopkins', 'Stanley Hopkins', 'Holmes', 'Hopkins', 'Holmes', 'Watson', 'Stanley Hopkins', 'Peter Carey', 'Woodman', 'Holmes', 'Peter Carey', 'Peter', 'Holmes', 'Peter', 'Holmes', 'Slater', 'Peter Carey', 'Tuesday Peter Carey', 'Holmes', 'Hopkins', 'Holmes', 'Peter Carey', 'Holmes', 'Holmes', 'Stanley Hopkins', 'Hopkins', 'Holmes', 'Holmes', 'Stanley Hopkins', 'Holmes', 'Sherlock Holmes', 'Holmes', 'Carey', 'Holmes', 'Stanley Hopkins', 'Watson', 'Hopkins', 'Stanley Hopkins', 'Peter Carey', 'Stanley Hopkins', 'Holmes', 'Hopkins', 'Watson', 'Hopkins', 'Hopkins', 'Holmes', 'Hopkins', 'Stanley Hopkins', 'Peter Carey', 'Hopkins', 'John Hopley Neligan', 'Holmes', 'Hopkins', 'Dawson', 'Hopkins', 'Holmes', 'Neligan', 'Peter Carey', 'Dawson', 'Peter Carey', 'Peter Carey', 'Peter Carey', 'Hopkins', 'Hopkins', 'Hopkins', 'Holmes', 'Watson', 'Holmes', 'Watson', 'Stanley Hopkins', 'Watson', 'Stanley H

In [72]:
def IOB_to_tree(iob_tagged):
    """
    Say input output, what the function does.
    """
    root = nltk.Tree('S', [])
    for token in iob_tagged:
        if token[2] == 'O':
            root.append((token[0], token[1]))
        else:
            try:
                if root[-1].label() == token[2]:
                    root[-1].append((token[0], token[1]))
                else:
                    root.append(Tree(token[2], [(token[0], token[1])]))
            except:
                root.append(Tree(token[2], [(token[0], token[1])]))
    return root

#reference: https://stackoverflow.com/questions/27629130/chunking-stanford-named-entity-recognizer-ner-outputs-from-nltk-format - how to cite it??


(S
  (PERSON Andrew/NNP)
  is/VBZ
  part/NN
  of/IN
  the/DT
  (ORGANIZATION Republican/NNP Party/NNP)
  in/IN
  (LOCATION Dallas/NNP))


In [56]:
sent = "Arthur Baba and Mesi Baba is from New York and he likes to eat at Mama's Little Bakery in Autumn."

In [80]:
tok = nltk.tokenize.word_tokenize(sent)
pos = nltk.pos_tag(tok)
ner = st.tag(tok)
new = [[name, pos, tp[1]] for (name,pos), tp in zip(pos, ner)]
new

[['Arthur', 'NNP', 'PERSON'],
 ['Baba', 'NNP', 'PERSON'],
 ['and', 'CC', 'O'],
 ['Mesi', 'NNP', 'PERSON'],
 ['Baba', 'NNP', 'PERSON'],
 ['is', 'VBZ', 'O'],
 ['from', 'IN', 'O'],
 ['New', 'NNP', 'LOCATION'],
 ['York', 'NNP', 'LOCATION'],
 ['and', 'CC', 'O'],
 ['he', 'PRP', 'O'],
 ['likes', 'VBZ', 'O'],
 ['to', 'TO', 'O'],
 ['eat', 'VB', 'O'],
 ['at', 'IN', 'O'],
 ['Mama', 'NNP', 'ORGANIZATION'],
 ["'s", 'POS', 'ORGANIZATION'],
 ['Little', 'JJ', 'ORGANIZATION'],
 ['Bakery', 'NNP', 'ORGANIZATION'],
 ['in', 'IN', 'O'],
 ['Autumn', 'NNP', 'DATE'],
 ['.', '.', 'O']]

In [59]:
tr = IOB_to_tree(new)
print(tr)

(S
  (PERSON Arthur/NNP Baba/NNP)
  and/CC
  (PERSON Mesi/NNP Baba/NNP)
  is/VBZ
  from/IN
  (LOCATION New/NNP York/NNP)
  and/CC
  he/PRP
  likes/VBZ
  to/TO
  eat/VB
  at/IN
  (ORGANIZATION Mama/NNP 's/POS Little/JJ Bakery/NNP)
  in/IN
  (DATE Autumn/NNP)
  ./.)


### 

In [69]:
person = []
location = []
time = []

for chunk in tr:
    if hasattr(chunk, 'label'):
        entity = " ".join(part[0] for part in chunk.leaves())
        if chunk.label() == "PERSON":
            person.append(entity)
        elif chunk.label() == "LOCATION":
            location.append(entity)
        elif (chunk.label() == "DATE") or (chunk.label() == "TIME"):
            time.append(entity)
        #print(chunk.label(), entity)
print(f"characters: {person}")  
print(f"locations: {location}")
print(f"dates / times: {time}")

characters: ['Arthur Baba', 'Mesi Baba']
locations: ['New York']
dates / times: ['Autumn']
